# Focus stacking with image bunches

Example of focus stacking with automatic alignment and luminosity balance, and partial stacking of image bunches before the final stacking.

In [ ]:
from focus_stack import *

In [ ]:
def make_job(x, y):
    job = StackJob("job", x + y, input_path="src")
    job.add_action(AlignLayers("align", border_mode='BORDER_REPLICATE'))
    job.add_action(BalanceLayersLumi("balance", i_min=150, i_max=65385, mask_size=0))
    job.add_action(FocusStackBunch("bunches", PyramidStack(), exif_path="src", frames=17, overlap=2, denoise=0))
    job.add_action(FocusStack("stack", PyramidStack(), exif_path="src", postfix='_pyr', denoise=0))
    return job

In [ ]:
for x in ["E:/Focus stacking/2025-01-11 - Emiptera/"]:
    for y in ['A', 'B']:
        make_job(x, y).run()
